In [10]:
import requests
import time
import json

# helper functions for making queries

AUTH_TOKEN = "03AKH6MRGfsxcnXXHz42b-rVmvIdqyihro1nLGOC7M7__kVGYhi_RrRWbmly-9Gw9WjVJBGcC56pO9phaU2J4N4d19wcdWMwEoMbsOUtjwOBAd5zMNvCgJ3lqCSBD5TjUQbceWZ9_NEhzPhdcvOwCQX43oWccNJlb8K-cONdhLUfz3ATAIo1DwJJ7xJ1-8zBWXEk8KSUV_kJv8Dc7X0jXHP-6stRkBqCy7vb_2N6iaBkZiN6arjmAv_dLAuEAociG-omwAHY-SObhs7q2SBGRHRs1FEW-J5CmEN6YSavK5XLH8TbfJ8pwhjNisNkk6B9eZTOSrvUj5FFVsdpm_JRz93-dm-RCKVuYY7vU32kitQpvLXHIJAEqttQKFBwmuo8I0DNDMpHhREUGfamPnB5a0aCeVzy4P7aLt4iSuZgCNKv-9mPMY7lYgvdBHxohxzKipY5izVfnTNLuHGPyEuFndiDhWcqv5LVt6HItwJibkWnOFiTStqW9kBSzjmkbBiuogzvZPdThH4LHxhqWm4i4LNNwnjfPe3ndyDw"
SLEEP_T = 2

postQueryEndpoint = "https://data.stackexchange.com/query/save/1/1727672"
getQueryEndpoint = "https://data.stackexchange.com/query/job/jobId"


def saveAsJson(fileName, content):
    # Serializing json
    json_object = json.dumps(content, indent=4)
 
    # Writing to sample.json
    with open(fileName, "w") as outfile:
        outfile.write(json_object)

def makeQuery(query, fileName=None):
    query = {
        "sql": query,
        "g-recaptcha-response": AUTH_TOKEN
    }

    # create initial query
    response = requests.post(postQueryEndpoint, data = query)
    response = response.json()

    print(response)

    running = response.get("running")
    getEndpoint = getQueryEndpoint

    # if this is a larger query, modify the get endpoint
    if running:
        getEndpoint = getEndpoint.replace("jobId", response.get("job_id"))

    # continue checking every interval for results
    while running:
        time.sleep(SLEEP_T)
        response = requests.get(getEndpoint, params = {"_": time.time()}).json()
        running = response.get("running")

    # save content to json if fileName provided
    content = response.get("resultSets")
    if fileName: saveAsJson(fileName, content)

    return content


In [ ]:
# making queries

makeQuery(
""" 
SELECT * FROM PostHistory WHERE UserId = 500;
""",
"test.json"
)